<a href="https://colab.research.google.com/github/Alicia-LB/Music-or-Travel/blob/main/Travel_Insurance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Travel Claim Insurance Predictions 
-Alicia Lizarraga
* DataSet Source: https://www.kaggle.com/mhdzahier/travel-insurance/version/1

###DataSet Description: 


###Objective:

##Importing & Mounting 

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
filename = '/content/travel insurance.csv.zip'
df = pd.read_csv(filename)
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Gender,Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,F,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,NaN,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,NaN,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,NaN,41


##EDA: Inspect Data

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63326 entries, 0 to 63325
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                63326 non-null  object 
 1   Agency Type           63326 non-null  object 
 2   Distribution Channel  63326 non-null  object 
 3   Product Name          63326 non-null  object 
 4   Claim                 63326 non-null  object 
 5   Duration              63326 non-null  int64  
 6   Destination           63326 non-null  object 
 7   Net Sales             63326 non-null  float64
 8   Commision (in value)  63326 non-null  float64
 9   Gender                18219 non-null  object 
 10  Age                   63326 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 5.3+ MB


In [24]:
df.isna().sum()

Agency                      0
Agency Type                 0
Distribution Channel        0
Product Name                0
Claim                       0
Duration                    0
Destination                 0
Net Sales                   0
Commision (in value)        0
Gender                  45107
Age                         0
dtype: int64

* The column Gender may impact audience target for which sex might file a claim for travel insurance, but in case of the objective there's not too much relation in how Gender affects the socioeconomic reality of the claim being filed. As well it seems it may have been an optional choice to clients to volunteer their Gender information. So in this case with so many missing, we will drop the column

In [25]:
df.drop(columns = 'Gender', inplace = True)
df.head()

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
0,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,81
1,CBH,Travel Agency,Offline,Comprehensive Plan,No,186,MALAYSIA,-29.0,9.57,71
2,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,65,AUSTRALIA,-49.5,29.70,32
3,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,60,AUSTRALIA,-39.6,23.76,32
4,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,No,79,ITALY,-19.8,11.88,41


In [26]:
df.duplicated().sum()

8670

In [27]:
df = df.drop_duplicates(keep = False)
df.duplicated().sum()

0

In [29]:
data_types = df.dtypes
str_cols = data_types[ data_types=='object'].index
str_cols

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Claim', 'Destination'],
      dtype='object')

* From a glance it doesn't seem like there are any words or phrases that are inconsistent or misspelled

In [30]:
for col in str_cols:
    print(f'- {col}:')
    print(df[col].value_counts(dropna=False))
    print("\n\n")

- Agency:
EPX    25914
CWT     7719
C2B     6901
JZI     5596
SSI      875
LWC      673
RAB      517
TST      428
ART      317
KML      252
JWT      247
CCR      159
CBH       97
TTW       86
CSR       84
ADM       82
Name: Agency, dtype: int64



- Agency Type:
Travel Agency    35494
Airlines         14453
Name: Agency Type, dtype: int64



- Distribution Channel:
Online     49003
Offline      944
Name: Distribution Channel, dtype: int64



- Product Name:
Cancellation Plan                       13631
2 way Comprehensive Plan                10049
Rental Vehicle Excess Insurance          7719
Basic Plan                               4748
Bronze Plan                              3312
1 way Comprehensive Plan                 2234
Silver Plan                              2036
Value Plan                               1877
Annual Silver Plan                       1034
Ticket Protector                          875
Travel Cruise Protect                     427
Gold Plan                       

In [35]:
df.describe()

,Duration,Net Sales,Commision (in value),Age
count,49947.000000,49947.000000,49947.000000,49947.000000
mean,54.066991,43.505300,11.054156,39.928604
std,108.678148,50.569755,21.164442,13.007587
min,-2.000000,-389.000000,0.000000,0.000000
25%,12.000000,19.800000,0.000000,34.000000
50%,27.000000,29.700000,0.000000,36.000000
75%,60.000000,53.500000,12.250000,45.000000
max,4881.000000,810.000000,283.500000,118.000000


In [54]:
df.loc[df['Net Sales'] == -389, :]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
16361,C2B,Airlines,Online,Annual Silver Plan,No,364,SINGAPORE,-389.0,97.25,27


In [44]:
df.loc[df['Duration'] == -2, :]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
14943,JZI,Airlines,Online,Basic Plan,No,-2,BANGLADESH,22.0,7.7,118


In [45]:
df = df.drop(index = 14943)

In [46]:
df.loc[df['Age'] == 0, :]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
41993,TST,Travel Agency,Offline,Travel Cruise Protect,No,94,THAILAND,30.0,10.5,0
52109,TST,Travel Agency,Offline,Travel Cruise Protect,No,94,THAILAND,-30.0,10.5,0


In [56]:
df.loc[df['Age'] == 1, :]

,Agency,Agency Type,Distribution Channel,Product Name,Claim,Duration,Destination,Net Sales,Commision (in value),Age
17021,JZI,Airlines,Online,Value Plan,No,2,MALAYSIA,26.0,9.10,1
57555,C2B,Airlines,Online,Silver Plan,No,10,SINGAPORE,46.8,11.70,1
57748,C2B,Airlines,Online,Silver Plan,No,33,SINGAPORE,74.4,18.60,1
60234,C2B,Airlines,Online,Bronze Plan,No,12,SINGAPORE,16.0,4.00,1
63211,C2B,Airlines,Online,Bronze Plan,No,56,SINGAPORE,86.5,21.63,1


In [50]:
df = df.drop(index = 41993)

In [51]:
df = df.drop(index = 52109)

In [52]:
n_unique = df.nunique()
n_unique[n_unique>2]

Agency                    16
Product Name              26
Duration                 454
Destination              149
Net Sales               1135
Commision (in value)    1030
Age                       88
dtype: int64

In [55]:
df.describe()

,Duration,Net Sales,Commision (in value),Age
count,49944.000000,49944.000000,49944.000000,49944.000000
mean,54.066514,43.507473,11.054246,39.928640
std,108.680829,50.570076,21.165072,13.000831
min,-1.000000,-389.000000,0.000000,1.000000
25%,12.000000,19.800000,0.000000,34.000000
50%,27.000000,29.700000,0.000000,36.000000
75%,60.000000,53.612500,12.250000,45.000000
max,4881.000000,810.000000,283.500000,118.000000
